# FinanceGPT: Using RAG Based LLM

In [2]:
import os
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

MONGODB_CONNECTION_STRING = os.getenv("MONGODB_URI")
MONGODB_DATABASE = os.getenv("MONGODB_DATABASE")
MONGODB_COLLECTION = os.getenv("MONGODB_COLLECTION")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


Python-dotenv could not parse statement starting at line 3


## Store chunks in MongoDB database

In [4]:
import os
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient(MONGODB_CONNECTION_STRING)
db = client[MONGODB_DATABASE]
collection = db[MONGODB_COLLECTION]

# Function to chunk text by sentence
def chunk_by_sentence(text):
    sentences = []
    tmp_sentence = ""
    for char in text:
        if char in [".", "!", "?"]:
            if tmp_sentence:  # Check if tmp_sentence is not empty
                sentences.append(tmp_sentence.strip())
            tmp_sentence = ""
        else:
            tmp_sentence += char
    if tmp_sentence:
        sentences.append(tmp_sentence.strip())
    return sentences

# Path to the directory containing cleaned text files
clean_text_dir = '/Users/mrinoyb2/git/FinanceGPT/data/pf_wiki_texts'

# Initialize a counter for unique MongoDB document IDs
doc_id = 0

# Iterate through each cleaned text file
for file_name in os.listdir(clean_text_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(clean_text_dir, file_name)

        # Read the cleaned text from the file
        with open(file_path, 'r') as file:
            cleaned_text = file.read()

        # Chunk the text
        chunks = chunk_by_sentence(cleaned_text)

        # Store chunks in MongoDB
        for chunk in chunks:
            # Create a document for each chunk
            document = {"_id": doc_id, "text": chunk}
            # Insert the document into the collection
            collection.insert_one(document)
            # Increment the doc_id for the next document
            doc_id += 1

print(f"Total chunks stored in MongoDB: {doc_id}")


Total chunks stored in MongoDB: 97


## Implement RAG

### Create word embeddings

In [5]:
from sentence_transformers import SentenceTransformer
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient(MONGODB_CONNECTION_STRING)
db = client[MONGODB_DATABASE]
chunks_collection = db[MONGODB_COLLECTION]

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to update documents with embeddings
def update_documents_with_embeddings():
    for document in chunks_collection.find():
        # Generate embedding
        embedding = model.encode(document['text'], convert_to_tensor=False)
        # Update document with embedding
        chunks_collection.update_one({'_id': document['_id']}, {'$set': {'embedding': embedding.tolist()}})

# Uncomment the following line to run the embedding update
update_documents_with_embeddings()

# Check the first document to see if the embedding was added
print(chunks_collection.find_one())


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'_id': 0, 'text': 'rpersonalfinanceendoflifeplanningopen menuview page historyview page sourcecopy url introductionwhat you need to dorelated articlesintroductionfrom time to time the personal finance community is faced with a request for help from an person settling an estate or facing the near term death of a family member often it is clear that very little planning has been done and there is little to be done at that point rather than regretting the lost opportunity these requests can serve as a wake up call for all of us to get our affairs in orderthere are many concerns to address at end of life including your spiritual affairs your personal relationships and wrapping up any unfinished business or goals these are out of scope for a finance discussion but being well positioned in your financial plans can permit use of remaining time for addressing these concernswhat you need to dowhen planning for endoflife make sure youve covered the followinghavethe conversation communicate your

### Semantic search retrieval

In [7]:
from sentence_transformers import SentenceTransformer
import pymongo
import numpy as np
from scipy.spatial.distance import cosine

# Connect to MongoDB
client = pymongo.MongoClient(MONGODB_CONNECTION_STRING)
db = client[MONGODB_DATABASE]
chunks_collection = db[MONGODB_COLLECTION]

# Function to perform semantic search
def semantic_search(query, top_k=5):
    # Generate query embedding
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=False)
    
    # Retrieve all embeddings from MongoDB and calculate similarity
    similarities = []
    for document in chunks_collection.find():
        doc_embedding = np.array(document['embedding'])
        similarity = 1 - cosine(query_embedding, doc_embedding)  # Higher score means more similar
        similarities.append((document['_id'], similarity, document['text']))
    
    # Sort by similarity score in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Return top_k most similar documents
    return similarities[:top_k]

# Example usage
query = "Give me best practices for portfolio management"
results = semantic_search(query)
for idx, (doc_id, similarity, text) in enumerate(results, start=1):
    print(f"Result {idx} (Score: {similarity:.3f}): {text}...")

Result 1 (Score: 0.515): rpersonalfinancefinancialadvisorsopen menuview page historyview page sourcecopy url should i use a financial advisorplannerprobably notwell maybewhat does feeonly meanwhich types of complex situationswhich specific credentials are best if i am looking for general personal finance adviceresources to find a financial advisorplannerchecking credentialsfinancial issues of a legal natureadditional readingshould i use a financial advisorplannerprobably notmany fas are paid differently depending on what you do with your money so they will inevitably be biased in favor of investments that maximize their commissions  this is especially true of financial advisors associated with fullservice brokerages insurance companies and fund companies focused on active management  for most people investing isnt more complicated than picking an asset allocation and finding lowcost indexfunds so the best fa in the world will just repeat the same advice we give herewell maybein complex

## Setting up Gemini Pro

In [8]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.

genai.configure(api_key=GEMINI_API_KEY)

In [10]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


## Connect LLM model

In [11]:
# Function to generate an answer using RAG enabled LLama2 from Replicate
def generate_RAG_answer(question, max_context_length=1000):
    # Assume semantic_search is defined and returns relevant context as a single string
    context_results = semantic_search(question, top_k=1)
    context = context_results[0][2]  # Get the text of the top result
    # Truncate context if it exceeds the maximum length
    if len(context) > max_context_length:
        context = context[:max_context_length]

    rag_prompt = f"[INST]\nQuestion: {question}\nContext: {context}\n[/INST]"
    print(rag_prompt)

    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(rag_prompt)
    # return text part of the response
    return response.text

# Function to generate an answer using LLama2 from Replicate
def generate_non_RAG_answer(question):
    # Assume semantic_search is defined and returns relevant context as a single string
    non_rag_prompt = f"[INST]\nQuestion: {question}\n[/INST]"  # Fallback in case no context is found
    print(non_rag_prompt)
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(non_rag_prompt)
    return response.text

In [18]:
# Example query
query = "Best practices for repaying loans"
rag_answer = generate_RAG_answer(query)

print(rag_answer)

[INST]
Question: Best practices for repaying loans
Context: rpersonalfinancestudentloansopen menuview page historyview page sourcecopy url a few guidelines for recent graduates with student loan debtrecent graduates should be aware of all their student loan repayment optionsrepaying student loansinformative postsrelated subredditshelpful subredditsa few guidelines for recent graduates with student loan debtlive frugallyspend as little as possible you owe a lot of money to something act like it if you dont grab it by the horns now it could haunt you for years to comecreate a budgetvisit ourbudgeting wikiuse ourtools wikithis subreddit also lovesynaband mintstick to your budgetallocate money to andira or 401kif possiblethis will depend on your interest rates generally if the interest is lower than 4 you could benefit more from putting money in an investment account remember putting money towards a 6 loan is a guaranteed 6 return another thing to remember is that if your company matches 4

In [19]:
# Example query
query = "Best practices for repaying loans"
non_rag_answer = generate_non_RAG_answer(query)

print(non_rag_answer)

[INST]
Question: Best practices for repaying loans
[/INST]
**Best Practices for Repaying Loans**

**1. Prioritize High-Interest Debt:**
   - Focus on paying off loans with the highest interest rates first to minimize interest charges.

**2. Make Extra Payments:**
   - If possible, make extra payments above the minimum required to accelerate repayment and reduce the total interest paid.

**3. Use Debt Consolidation:**
   - Consider consolidating multiple loans into a single loan with a lower interest rate to simplify repayment and reduce monthly payments.

**4. Refinance:**
   - Explore refinancing options to reduce interest rates or extend repayment terms, potentially lowering monthly payments or saving money on interest.

**5. Negotiate with Lenders:**
   - If experiencing financial difficulties, contact lenders to discuss repayment plans or loan modifications to avoid default.

**6. Seek Credit Counseling:**
   - Consider working with a non-profit credit counseling agency for guidanc

## Evaluating RAG vs Non-RAG Approach

In order to quantify this, I will take the following steps:
1. Think of a query which has a well defined answer in the book.
2. Find the true answer to a query from the actual source (book pdf). 
3. Then pass the query through the RAG based LLM and the regular LLM.
4. Save both generated answers along with the true answer and find word embeddings for each. 
5. Finally, compare the word embeddings of the true answer with the RAG vs Non-RAG based LLM word embeddings using cosine similarity. 

Following these steps will help quantify the performace and accuracy of information in the two answers.

In [33]:
query = "Best practices for repaying loans"

# True Answer from the Huberman Lab Newsletter
true_answer = """"""

### Implementing functions to evaluate performance. 

In [ ]:
import torch

def get_embedding(text):
    """
    Generate an embedding for a given text.

    Args:
    - text (str): The input text.
    
    Returns:
    - The sentence embedding.
    """
    # Load the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    # Generate the sentence embeddings
    embeddings = model.encode(text, convert_to_tensor=False)
    return embeddings


def calculate_cosine_similarity(embedding1, embedding2):
    """
    Calculate the cosine similarity between two embeddings.

    Args:
    - embedding1 (torch.Tensor): The first embedding.
    - embedding2 (torch.Tensor): The second embedding.

    Returns:
    - The cosine similarity score.
    """
    # Calculate the cosine similarity
    similarity = 1 - cosine(embedding1, embedding2)
    return similarity


def calculate_similarity_scores(true_answer, rag_answer, non_rag_answer):
    """
    Calculate the cosine similarity scores between the true answer and both RAG-based and non-RAG-based answers.

    Args:
    - true_answer (str): The true answer text.
    - rag_answer (str): The RAG-based model's answer text.
    - non_rag_answer (str): The non-RAG-based model's answer text.

    Returns:
    - A dictionary with cosine similarity scores.
    """
    # Convert the answers to embeddings
    true_answer_embedding = get_embedding(true_answer)
    rag_answer_embedding = get_embedding(rag_answer)
    non_rag_answer_embedding = get_embedding(non_rag_answer)
    
    # Calculate cosine similarity scores
    rag_similarity = calculate_cosine_similarity(true_answer_embedding, rag_answer_embedding)
    non_rag_similarity = calculate_cosine_similarity(true_answer_embedding, non_rag_answer_embedding)

    
    # Return the scores
    return {
        "RAG Similarity Score": rag_similarity,
        "Non-RAG Similarity Score": non_rag_similarity
    }


# Calculate the similarity scores
similarity_scores = calculate_similarity_scores(true_answer, rag_answer, non_rag_answer)
print(similarity_scores)
